In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *


In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/cellphoneaccessory"

## Dataset

In [3]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
bs=52 # 52 - Jeremey, 20 - default

In [5]:
bptt= 70 #70 - Jeremey, 35 - default

In [6]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [7]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
train_tokens.shape, valid_tokens.shape

((52, 349220), (52, 38606))

In [8]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [9]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [10]:
x.shape,y.shape

((52, 70), (52, 70))

In [11]:
x,y

(array([[  18,   19,   20,   17, ...,   13,  128,   13,   10],
        [ 109,   10,   53,    6, ...,   45,  501,  257,    3],
        [   3,    2,   46,   26, ...,   44,   33, 2451,   10],
        [   6,    5,  142,   53, ...,    2,  604,    6,   25],
        ...,
        [   2,   35,  746,    5, ...,  534,    3,    5,   56],
        [ 113,   22,    9, 7475, ...,   38,    2,   70,   12],
        [   7,    8,   26,   40, ...,    3,   18,   19,   20],
        [   3,    2,  473,  134, ...,  515,  162,   80,   12]]),
 array([[  19,   20,   17,    2, ...,  128,   13,   10,   13],
        [  10,   53,    6,  159, ...,  501,  257,    3,    2],
        [   2,   46,   26,  167, ...,   33, 2451,   10,   56],
        [   5,  142,   53,    9, ...,  604,    6,   25,   72],
        ...,
        [  35,  746,    5,  128, ...,    3,    5,   56,  422],
        [  22,    9, 7475,  612, ...,    2,   70,   12,   23],
        [   8,   26,   40,  547, ...,   18,   19,   20,   17],
        [   2,  473,  134, 

In [12]:
x.shape,y.shape

((52, 70), (52, 70))

In [13]:
train_tokens.shape

(52, 349220)

## pre-trained AWD-LSTM

In [14]:
PATH_AWD_LSTM='/home/kirana/Documents/phd/data/pre-trained/awd_lstm/'

In [15]:
ls {PATH_AWD_LSTM}

bwd_wt103_enc.h5  fwd_wt103_enc.h5  itos_wt103.pkl  wt103_60002/
bwd_wt103.h5      fwd_wt103.h5      wt103_238642/   wt103_tiny.tgz


In [16]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103.h5', map_location=lambda storage, loc: storage)

In [17]:
wgts

OrderedDict([('0.encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('0.encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                

In [18]:
wgts = torch.load(f'{PATH_AWD_LSTM}/fwd_wt103_enc.h5', map_location=lambda storage, loc: storage)

In [19]:
wgts

OrderedDict([('encoder.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                        1.9580e-02,  1.8548e-01],
                      [ 1.4854e-05, -2.3424e-05,  1.9693e-05,  ...,  2.1349e-05,
                        2.1776e-05, -1.2394e-05],
                      [ 1.8070e-01,  1.5874e+00, -1.1738e-01,  ..., -4.5935e-02,
                       -8.1352e-02,  1.8054e-01],
                      ...,
                      [-1.8595e-03, -6.8529e-03,  1.6999e-03,  ...,  1.7039e-03,
                        4.1632e-03, -1.3171e-03],
                      [-2.3120e-03, -6.9001e-03,  1.8772e-03,  ...,  5.0309e-04,
                        4.6596e-03, -2.5850e-03],
                      [-2.2463e-03, -9.1512e-03,  1.3927e-03,  ...,  1.2296e-03,
                        5.8085e-03, -1.8940e-03]])),
             ('encoder_with_dropout.embed.weight',
              tensor([[-1.2274e-01,  2.7886e-01, -3.8850e-01,  ..., -1.0404e-01,
                    

In [20]:
itos2=pickle.load(open(f'{PATH_AWD_LSTM}/itos_wt103.pkl','rb'))
stoi2 = collections.defaultdict(lambda: -1, { v: k for k, v in enumerate(itos2) })

In [21]:
wgts['encoder.weight'].shape

torch.Size([238462, 400])

In [22]:
enc_wgts = wgts['encoder.weight'].numpy() # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [23]:
enc_wgts.shape, len(itos)

((238462, 400), 30238)

In [24]:
n_emb=enc_wgts.shape[1]

In [25]:
new_w = np.zeros((len(itos),n_emb), dtype=np.float32) # shape: (60002, 400)

for i, w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r >= 0 else row_m

In [26]:
len(set(itos).difference(set(itos2)))

9402

## Model Architecture

In [27]:
from torch import nn
import torch
from torch.autograd import Variable

In [28]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers=2
dropout=0.5
wd=1e-7

In [29]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [30]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [31]:
device="cuda:0"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [32]:
new_w.shape

(30238, 400)

In [33]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([30238, 400]), torch.Size([30238, 400]))

In [34]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([30238, 400]), 400, 30238)

### Test if model forward works

In [35]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([30238, 400]), torch.Size([30238, 400]))

In [36]:
x.shape

(52, 70)

In [37]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [38]:

model=model.to(device)

## Learner

In [39]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1

    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.gen_hidden()
        #for k,i in enumerate(range(0,n_batch,self.bptt)):
        n_batch=iterator.shape[1]
        while i<n_batch-bptt:
            if mode_train:
                cust_bptt=self.bptt if np.random.random() < 0.95 else self.bptt//np.random.randint (2,4)
            else:
                cust_bptt=bptt
            seq_len=min(cust_bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
            k=k+1
            i=i+seq_len
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [40]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [41]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

4989

In [59]:
model.freeze_embedding()

In [43]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [49]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,10000,0.25)

In [45]:
model.n_inp

30238

In [46]:
len(train_tokens), len(valid_tokens)

(52, 52)

In [47]:
df_train.shape

(155700, 5)

In [50]:
learner.run_epochs(train_tokens,valid_tokens,5)

Epoch:0 Loss:5.044906149873117
Epoch:0 Loss:5.044906149873117 Accuracy:0.18903325404318533 Loss:4.282061222893357 Accuracy:0.23383709174427061
Epoch:1 Loss:4.526315976247609
Epoch:1 Loss:4.526315976247609 Accuracy:0.22384560171433512 Loss:4.143852847890282 Accuracy:0.26221107503571656
Epoch:2 Loss:4.4497637319444285
Epoch:2 Loss:4.4497637319444285 Accuracy:0.2324531795089402 Loss:4.102019507309486 Accuracy:0.2675963884689414
Epoch:3 Loss:4.414959174364445
Epoch:3 Loss:4.414959174364445 Accuracy:0.23626207181826414 Loss:4.077250600942899 Accuracy:0.2702838100506476
Epoch:4 Loss:4.393212403801917
Epoch:4 Loss:4.393212403801917 Accuracy:0.23870510033140763 Loss:4.058779211961639 Accuracy:0.27272293223118826


In [52]:
for param_group in optimizer.param_groups:
    print(param_group['lr'])

0.001


In [53]:
for param_group in optimizer.param_groups:
    param_group['lr']=5e-4

In [54]:
learner.run_epochs(train_tokens,valid_tokens,2)

Epoch:0 Loss:4.360720880731833
Epoch:0 Loss:4.360720880731833 Accuracy:0.24227506387270908 Loss:4.0287062161198115 Accuracy:0.27611087903569703
Epoch:1 Loss:4.348983817733462
Epoch:1 Loss:4.348983817733462 Accuracy:0.24362821396731738 Loss:4.016934303969091 Accuracy:0.27768244782290313


In [60]:
for param_group in optimizer.param_groups:
    param_group['lr']=1e-4

In [61]:
learner.run_epochs(train_tokens,valid_tokens,2)

Epoch:0 Loss:4.330928662887443
Epoch:0 Loss:4.330928662887443 Accuracy:0.24560643849868333 Loss:4.002791845214345 Accuracy:0.2794270278537771
Epoch:1 Loss:4.324945534839763
Epoch:1 Loss:4.324945534839763 Accuracy:0.24615321052405312 Loss:3.9981312448878903 Accuracy:0.27978651466257126


In [62]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict')
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm')

In [63]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights_freeze','wb'))

In [64]:
model.unfreeze_embedding()

In [65]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [66]:
for param_group in optimizer.param_groups:
    param_group['lr']=5e-4

In [67]:
learner.run_epochs(train_tokens,valid_tokens,1)

Epoch:0 Loss:3.98885220368331
Epoch:0 Loss:3.98885220368331 Accuracy:0.28449018054425856 Loss:3.5798200098442727 Accuracy:0.32061139024969887


In [68]:
np.exp(3.57)

35.51659315162847

In [69]:
learner.run_epochs(train_tokens,valid_tokens,2)

Epoch:0 Loss:3.817732463857879
Epoch:0 Loss:3.817732463857879 Accuracy:0.29453458813161904 Loss:3.4959105774625026 Accuracy:0.32534454251806877
Epoch:1 Loss:3.748959439836508
Epoch:1 Loss:3.748959439836508 Accuracy:0.2986992632972024 Loss:3.4480200891269748 Accuracy:0.32821594527542264


In [70]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights_freeze','wb'))

In [71]:
learner.run_epochs(train_tokens,valid_tokens,3)

Epoch:0 Loss:3.7068574866299864
Epoch:0 Loss:3.7068574866299864 Accuracy:0.3014067806663943 Loss:3.4144628008999973 Accuracy:0.33059024902740103
Epoch:1 Loss:3.676902432195632
Epoch:1 Loss:3.676902432195632 Accuracy:0.30347215821703544 Loss:3.3901200458921235 Accuracy:0.3325896112217877
Epoch:2 Loss:3.654093524802672
Epoch:2 Loss:3.654093524802672 Accuracy:0.30511261368181575 Loss:3.369099552099155 Accuracy:0.3340689390627312


In [72]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights_unfreeze','wb'))

In [73]:
for param_group in optimizer.param_groups:
    param_group['lr']=7e-4

In [74]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict_nofreeze')
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm_nofreeze')

In [75]:
learner.run_epochs(train_tokens,valid_tokens,3)

Epoch:0 Loss:3.6463532880753693
Epoch:0 Loss:3.6463532880753693 Accuracy:0.30517356850167615 Loss:3.3609841917041425 Accuracy:0.3342683771236406
Epoch:1 Loss:3.6297835546240056
Epoch:1 Loss:3.6297835546240056 Accuracy:0.30645713239550265 Loss:3.345677702915863 Accuracy:0.3354450587568612
Epoch:2 Loss:3.614953364363164
Epoch:2 Loss:3.614953364363164 Accuracy:0.3076373416726818 Loss:3.332489339063475 Accuracy:0.3365594171352698


In [77]:
np.exp(3.61),np.exp(3.33)

(36.966052814822504, 27.938341703236507)

In [78]:
torch.save(model.state_dict(),f'{DATAPATH}/inter/varybptt_model_state_dict_unfreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/varybptt_learner_state_dict_unfreeze')

In [79]:
torch.save (model,f'{DATAPATH}/inter/varybptt_model_awd_lstm')
torch.save (optimizer,f'{DATAPATH}/inter/varybptt_optimizer_awd_lstm')
torch.save (learner,f'{DATAPATH}/inter/varybptt_learner_awd_lstm')

In [80]:
pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
import pickle
pickle.dump(pretrained_lm_weights,open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights_unfreeze','wb'))